### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 25 ноября 2019, 15:00   
**Штраф за опоздание:** -2 балла после 15:00 25 ноября, -4 балла после 15:00 2 декабря, -6 баллов после 15:00 9 декабря  -8 баллов после 15:00 16 декабря.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0919, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на [wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html) и [Speed Dating Data](https://cloud.mail.ru/public/8nHV/p6J7wY1y1)

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
# %load_ext pycodestyle_magic

In [39]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0,
                 criterion='gini', max_features=None):
        self.tree = dict()
        self.impurities = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise
    
    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return (l_s * (1 - np.sum((l_c / l_s) ** 2, axis=1)).reshape(-1, 1) +
                r_s * (1 - np.sum((r_c / r_s) ** 2, axis=1)).reshape(-1, 1)) / (l_s + r_s)

    def __entropy(self, l_c, l_s, r_c, r_s):
        return (l_s * (-np.sum((l_c / l_s) * np.log((l_c) / l_s), axis=1)).reshape(-1, 1) +
                r_s * (-np.sum((r_c / r_s) * np.log((r_c) / r_s), axis=1)).reshape(-1, 1)) / (l_s + r_s)

    def __misclass(self, l_c, l_s, r_c, r_s):
        return (l_s * (1 - np.max(l_c / l_s, axis=1)).reshape(-1, 1) +
                r_s * (1 - np.max(r_c / r_s, axis=1)).reshape(-1, 1)) / (l_s + r_s)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = list(range(n_feature))
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.ceil(np.sqrt(n_feature)))]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = list(range(n_feature))
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.ceil(np.log2(n_feature)))]

    def __get_feature_ids_N(self, n_feature):
        return list(range(n_feature))
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        x_sort, y_sort = self.__sort_samples(x, y)

        class_brd = np.where(y_sort[:-1] != y_sort[1:])[0] + 1

        if len(class_brd) == 0:
            return np.inf, None

        eq_el_count = class_brd - np.append([0], class_brd[:-1])
        one_hot_code = np.zeros((class_brd.shape[0], self.num_class))
        one_hot_code[np.arange(class_brd.shape[0]), y_sort[class_brd - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)

        l_c = np.cumsum(class_increments, axis=0)
        r_c = np.bincount(y, minlength=self.num_class) - l_c
        l_s = class_brd.reshape(l_c.shape[0], 1)
        r_s = y_sort.shape[0] - l_s

        gs = self.G_function(l_c, l_s, r_c, r_s)
        idx = np.argmin(gs)

        left_el_id = l_s[idx][0]
        return gs[idx], (x_sort[left_el_id - 1] + x_sort[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth):
        n_features = x.shape[1]
        bins = np.bincount(y)
        
        if (depth == self.max_depth \
            or y.shape[0] < self.min_samples_split \
            or np.unique(y).shape[0] == 1 \
            or bins.max() >= self.sufficient_share * y.shape[0]):
            
            self.tree[node_id] = (self.LEAF_TYPE, bins.argmax(), bins.max() / n_features)
        else:
            feature_ids = self.get_feature_ids(n_features)
            thresholds = np.array([self.__find_threshold(x[:, idx], y) for idx in feature_ids])
            best_thresh_id = thresholds[:, 0].argmin()
            
            impurity, threshold = thresholds[best_thresh_id]
            x_l, x_r, y_l, y_r = self.__div_samples(x, y, feature_ids[best_thresh_id], threshold)
            
            if node_id:
                impurities = self.impurities[(node_id - 1) // 2]
                self.feature_importances_[best_thresh_id] += \
                    impurities[0] - impurities[1][node_id % 2] * impurity
            else:
                self.feature_importances_[best_thresh_id] += 1 - impurity
            
            if y_l.shape[0] == 0 or y_r.shape[0] == 0:
                self.tree[node_id] = (self.LEAF_TYPE, bins.argmax(), bins.max() / n_features)
            else:
                self.tree[node_id] = (self.NON_LEAF_TYPE, best_thresh_id, threshold)
                self.impurities[node_id] = (impurity, np.array([y_l.shape[0], y_r.shape[0]]) / y.shape[0])
                self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1)
                self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)
            
    
    def fit(self, x, y):
        self.feature_importances_ = np.zeros(x.shape[1])
        
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)
        
        self.feature_importances_ /= self.feature_importances_.sum()

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [40]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [41]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [42]:
%time clf.fit(X_train, y_train)

CPU times: user 1.95 ms, sys: 0 ns, total: 1.95 ms
Wall time: 1.54 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [43]:
%time my_clf.fit(X_train, y_train)

CPU times: user 15 ms, sys: 0 ns, total: 15 ms
Wall time: 14.1 ms


## Проверка качества работы на wine

In [45]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8885003885003885

In [46]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.9440559440559441

In [47]:
my_clf.feature_importances_

array([0.04864194, 0.09754123, 0.        , 0.        , 0.05487926,
       0.        , 0.29003717, 0.        , 0.        , 0.17216971,
       0.        , 0.04136954, 0.29536115])

In [48]:
clf.feature_importances_

array([0.00442139, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.20573053, 0.        , 0.        , 0.36036196,
       0.        , 0.03260394, 0.39688217])

## Подготовка данных Speed Dating Data 

In [49]:
df = pd.read_csv('./data/speed-dating-experiment/Speed Dating Data.csv', encoding='latin1')

df = df.iloc[:, :97]
df = df.drop(['id'], axis=1)
df = df.drop(['idg'], axis=1)
df = df.drop(['condtn'], axis=1)
df = df.drop(['round'], axis=1)
df = df.drop(['position', 'positin1'], axis=1)
df = df.drop(['order'], axis=1)
df = df.drop(['partner'], axis=1)

df = df.drop(['age_o', 'race_o', 'pf_o_att', 
              'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o','met_o'], 
             axis=1)

df = df.dropna(subset=['age'])
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(19)
df = df.drop(['field'], axis=1)

df = df.drop(['undergra'], axis=1)

df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].fillna(df.mn_sat.min())

df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].fillna(df.tuition.min())

df = df.dropna(subset=['imprelig', 'imprace'])

df = df.drop(['from', 'zipcode'], axis=1)

df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(df.income.min())

df = df.dropna(subset=['date'])

df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(18)
df = df.drop(['career'], axis=1)

df = df.drop(['sports', 'tvsports', 'exercise', 'dining', 'museums', 
              'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv', 
              'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga'], axis=1)

df = df.drop(['expnum'], axis=1)

df.loc[:, 'temp_totalsum'] = \
    df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
    (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].T / 
     df.loc[:, 'temp_totalsum'].T).T * 100

df.loc[:, 'temp_totalsum'] = \
    df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] = \
    (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].T /
     df.loc[:, 'temp_totalsum'].T).T * 100

df = df.drop(['temp_totalsum'], axis=1)

for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i), 
            'intel{}_1'.format(i), 'fun{}_1'.format(i), 
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    
    df = df.drop(feat, axis=1)
    
df = df.drop(['wave'], axis=1)

df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                                 .drop(['gender'], axis=1)\
                                 .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                                   .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)\
                                   .dropna()
df_female.columns = df_female.columns + '_f'

df_final = df_male.join(df_female.set_index('iid_f'), on='pid', how='inner')
df_final = df_final.drop(['iid', 'pid', 'pid_f'], axis=1)

X = df_final.loc[:, df_final.columns != 'match'].to_numpy()
y = df_final.match.to_numpy()

In [50]:
df_final

,match,int_corr,samerace,age,field_cd,mn_sat,tuition,race,imprace,imprelig,...,sinc2_1_f,intel2_1_f,fun2_1_f,amb2_1_f,shar2_1_f,attr3_1_f,sinc3_1_f,fun3_1_f,intel3_1_f,amb3_1_f
3489,0,0.43,0,27.0,16.0,1450.0,26100.0,2.0,2.0,3.0,...,1.0,1.0,3.0,10.0,10.0,7.0,7.0,8.0,8.0,10.0
3498,0,0.46,0,28.0,8.0,1155.0,13258.0,2.0,8.0,9.0,...,1.0,1.0,3.0,10.0,10.0,7.0,7.0,8.0,8.0,10.0
3507,0,0.05,0,32.0,7.0,1430.0,26908.0,2.0,1.0,6.0,...,1.0,1.0,3.0,10.0,10.0,7.0,7.0,8.0,8.0,10.0
3516,0,0.44,0,25.0,10.0,1140.0,9168.0,2.0,7.0,1.0,...,1.0,1.0,3.0,10.0,10.0,7.0,7.0,8.0,8.0,10.0
3534,0,0.23,1,27.0,8.0,1360.0,26062.0,6.0,1.0,1.0,...,1.0,1.0,3.0,10.0,10.0,7.0,7.0,8.0,8.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8112,0,0.48,0,24.0,5.0,1450.0,26908.0,2.0,5.0,1.0,...,25.0,10.0,10.0,5.0,25.0,6.0,10.0,9.0,7.0,5.0
8178,0,-0.24,1,24.0,5.0,1320.0,25533.0,4.0,9.0,6.0,...,25.0,10.0,10.0,5.0,25.0,6.0,10.0,9.0,7.0,5.0
8200,0,0.12,1,23.0,5.0,1309.0,15162.0,4.0,3.0,8.0,...,25.0,10.0,10.0,5.0,25.0,6.0,10.0,9.0,7.0,5.0
8222,0,0.31,0,24.0,5.0,1430.0,26908.0,2.0,7.0,1.0,...,25.0,10.0,10.0,5.0,25.0,6.0,10.0,9.0,7.0,5.0


In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

In [52]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2, max_features='sqrt')
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы на Speed Dating Data 

In [53]:
%time clf.fit(X_train, y_train)

CPU times: user 19.2 ms, sys: 0 ns, total: 19.2 ms
Wall time: 18.2 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [54]:
%time my_clf.fit(X_train, y_train)

CPU times: user 18.4 ms, sys: 44 µs, total: 18.4 ms
Wall time: 17.8 ms


## Проверка качества работы на Speed Dating Data

In [55]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5306432038834952

In [56]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

/home/ayakovenko/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.45662100456621

## Задание 3

## Задание 4

In [62]:
col_names = df_final.loc[:, df_final.columns != 'match'].columns

In [73]:
print('MyDecisionTreeClassifier importances')
d = {}
for idx in np.argsort(-my_clf.feature_importances_)[:10]:
    d[col_names[idx]] = my_clf.feature_importances_[idx]

d

MyDecisionTreeClassifier importances


{'tuition': 0.29415787333753457,
 'age': 0.1777563236979994,
 'int_corr': 0.17427630023606414,
 'field_cd': 0.1566489282958818,
 'samerace': 0.07875024900151749,
 'imprace': 0.061196962092827885,
 'race': 0.05721336333817472,
 'attr1_1_f': 0.0,
 'exphappy_f': 0.0,
 'career_c_f': 0.0}

In [74]:
print('DecisionTreeClassifier importances')
d = {}
for idx in np.argsort(-clf.feature_importances_)[:10]:
    d[col_names[idx]] = clf.feature_importances_[idx]

d

DecisionTreeClassifier importances


{'int_corr': 0.0859416660662911,
 'attr2_1': 0.05422552053579554,
 'shar1_1_f': 0.053878758507275804,
 'income_f': 0.049955026864601015,
 'income': 0.028956468123186915,
 'attr2_1_f': 0.028794517787253886,
 'sinc1_1': 0.027073449585290184,
 'amb3_1_f': 0.02532633751424487,
 'race': 0.025228310952999936,
 'samerace': 0.025023711028150538}

## Задание 5

In [83]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': np.arange(1, 10),
    'min_samples_leaf': np.arange(1, 6)}

In [84]:
model = DecisionTreeClassifier()
random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=200, n_jobs=-1, scoring='f1')
random_search.fit(X, y)

/home/ayakovenko/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/home/ayakovenko/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 90 is smaller than n_iter=200. Running 90 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
/home/ayakovenko/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
                   estimator=DecisionTreeClassifier(class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort=False,
                                                    random_state=None,
                                                    splitter='b

In [85]:
random_search.best_params_

{'min_samples_leaf': 2, 'max_depth': 9, 'criterion': 'gini'}

In [86]:
f1_score(y_pred=random_search.best_estimator_.predict(X_test), y_true=y_test, average='macro')

0.7702299455853958